In [ ]:
import os, requests, zipfile, time, random
from pathlib import Path
from datetime import date, timedelta

BASE = Path("D:/desktop/weather_data/prism_raw")
ELEMENTS = ["ppt", "tmin", "tmax"]
YEARS = range(2023, 2025)


def download_one(element, dt):
    y = dt.year
    year_dir = BASE / element / str(y)
    year_dir.mkdir(parents=True, exist_ok=True)

    fname = f"{element}_{dt.strftime('%Y%m%d')}.tif"
    out_tif = year_dir / fname
    if out_tif.exists():
        return True

    # 1. 下载 zip
    zip_path = year_dir / (fname + ".zip")
    url = f"https://services.nacse.org/prism/data/get/us/4km/{element}/{dt:%Y%m%d}"

    try:
        r = requests.get(url, timeout=40)
        if r.status_code != 200:
            print("Bad status:", r.status_code, url)
            return False

        with open(zip_path, "wb") as f:
            f.write(r.content)

        # 2. 检查 zip 正确性
        if not zipfile.is_zipfile(zip_path):
            print("[WARN] Not a zip file:", zip_path)
            return False

        # 3. 解压 tif
        with zipfile.ZipFile(zip_path,"r") as zf:
            members = [n for n in zf.namelist() if n.endswith(".tif")]
            if not members:
                print("[WARN] Zip has no tif:", zip_path)
                return False
            with zf.open(members[0]) as src, open(out_tif,"wb") as dst:
                dst.write(src.read())

        return True

    except Exception as e:
        print("Download error:", url, e)
        return False

    finally:
        time.sleep(random.uniform(0.5, 1))  # 防止 PRISM 限频


def main():
    for y in YEARS:
        d = date(y, 1, 1)
        end = date(y, 12, 31)

        while d <= end:
            for element in ELEMENTS:
                ok = download_one(element, d)
                if not ok:
                    print("Failed:", element, d)
            d += timedelta(days=1)

        print("Finished year:", y)

main()

Finished year: 2023
Finished year: 2024
